In [1]:
println("Hello World!")

Hello World!


In [2]:
calinput = "../deps/src/CalculiX-cmake/src/fortran/calinput.f"

"../deps/src/CalculiX-cmake/src/fortran/calinput.f"

In [3]:
function pushcleanedargs!(arglist,inputstring)
    for arg in split(strip(inputstring),",")
        cleaned_arg = strip(arg)
        if cleaned_arg != ""
            push!(arglist,cleaned_arg);
        end
    end
end

pushcleanedargs! (generic function with 1 method)

In [10]:
function find_subroutine_args(file_name)
    activeline = false
    subroutineargs = []
    fil = open(file_name)
    
    for line in readlines(fil)
        if activeline & ismatch(r"\)",line)
            activeline = false          
            line_middle_part = split(split(line,"&")[2],")")[1]
            pushcleanedargs!(subroutineargs,line_middle_part)
        end
        if activeline
            line_end_part = split(line,"&")[2]
            pushcleanedargs!(subroutineargs,line_end_part)
        end
        if ismatch(r"subroutine",line)
            activeline = true
            line_end_part = split(line,"(")[2]
            pushcleanedargs!(subroutineargs,line_end_part)
        end
    end
    close(fil)
    return subroutineargs
end

find_subroutine_args (generic function with 1 method)

In [80]:
subargs = find_subroutine_args(calinput);

In [81]:
function findalldefinitions(file_name)
    activeline = false
    continuecomp = "empty"
    fil = open(file_name)
    output = Dict()
    keyw = ["logical" "character" "integer" "real"]
    for key in keyw
        output[key] = []
    end
    for line in readlines(fil)
        if activeline
            if ismatch(r"\&",line)
                #println(line)
                m = match(r"(\s*&\s+)(.*)",line)
                pushcleanedargs!(output[continuecomp],m.captures[2])
            else
                activeline = false
            end
        end
        for comp in keyw
            if ismatch(Regex(comp),line)
                
                #println(line)
                activeline = true
                m = match(r"(\s*\w+\s+)(.*)",line)
                pushcleanedargs!(output[comp],m.captures[2])
                continuecomp = comp
            end
        end
    end
    close(fil)
    return output
end

findalldefinitions (generic function with 1 method)

In [82]:
types = findalldefinitions(calinput);

In [98]:
function printccall(args,dictoftypes)
    print("(")
    for arg in args
        for key in keys(dictoftypes)
            if arg in dictoftypes[key]
                if key == "real"
                    print("Ptr{Float64}, ")
                elseif key == "integer"
                    print("Ptr{Int64}, ")
                elseif key == "logical"
                    print("Ptr{Bool}, ")
                elseif key == "character"
                    print("Ptr{ASCIIString}, ")
                end
            end
        end
    end
    print(")")
end

printccall (generic function with 1 method)

In [99]:
printccall(subargs,types)

(Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Float64}, Ptr{ASCIIString}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, Ptr{Int64}, )

In [94]:
Ptr{ASCIIString}

Ptr{ASCIIString}